In [2]:
from fastembed import TextEmbedding

## Q1. Embedding the query

In [10]:
from fastembed import TextEmbedding
model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_name)

In [11]:
user_question = "I just discovered the course. Can I join now?"
embeddings = list(embedding_model.embed([user_question]))

In [13]:
print('Answer embedding:', len(embeddings[0]))

Answer embedding: 512


In [14]:
print('Minimal value in the array:', min(embeddings[0]))

Minimal value in the array: -0.11726373885183883


## Q2. Cosine similarity with another vector

In [21]:
import numpy as np
np.linalg.norm(embeddings)

np.float64(1.0)

In [22]:
print(embeddings[0].dot(embeddings[0]))

1.0000000000000002


In [47]:
doc = 'Can I still join the course after the start date?'

doc_embedding = list(embedding_model.embed([doc]))[0]
res_embedding = embeddings
print(res_embedding[0].dot(doc_embedding))

0.9008528895674548


## 3. Ranking by cosine

In [48]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [63]:
document_list = []

for i, doc in enumerate(documents):
    text = doc['text']
    document_list.append(text)

In [73]:
new_doc_embedding = embedding_model.embed(document_list)
new_doc_embd_list = list(new_doc_embedding)
cos_res = [float(embeddings[0].dot(x)) for x in new_doc_embd_list]
cos_res

[0.7629684696540238,
 0.8182378150042889,
 0.8085397398734189,
 0.7133079015686243,
 0.7304499234333598]

In [74]:
max_cos = max(cos_res)
max_cos

0.8182378150042889

## 4. Ranking by cosine, version two

In [76]:
text_document_list = []

for i, doc in enumerate(documents):
    text = doc['question'] + doc['text']
    text_document_list.append(text)

text_doc_embedding = embedding_model.embed(text_document_list)
text_doc_embd_list = list(text_doc_embedding)

text_cos = [float(embeddings[0].dot(x)) for x in text_doc_embd_list]
text_cos

[0.8514543236908068,
 0.8436594159113068,
 0.8408287048502558,
 0.7755157969663908,
 0.8086007795043938]

In [78]:
max_cos = max(text_cos)
max_cos

0.8514543236908068

## 5. Selecting the embedding model

In [ ]:
TextEmbedding.list_supported_models()

In [80]:
model_handle = "BAAI/bge-small-en"

In [81]:
import json

for model in TextEmbedding.list_supported_models():
    if model["model"] == model_handle:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}


## Q6. Indexing with qdrant 

In [83]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [86]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [87]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = "BAAI/bge-small-en"

In [88]:
collection_name = "zoomcamp-hw"

In [89]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [90]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [91]:
points[0]

PointStruct(id=0, vector=Document(text='How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', model='BAAI/bge-small-en', options=None), payload={'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork', 'section': 'General course-related questions', 'question': 'How do I s

In [92]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [93]:
question = "I just discovered the course. Can I join now?"

In [94]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    limit=5,
    with_payload=True
)

In [99]:
p_score = []
for point in query_points.points:
    print(f'score:{point.score}')
    p_score.append(point.score)

score:0.8703172
score:0.86918855
score:0.86833113
score:0.8576106
score:0.85715395


In [100]:
max(p_score)

0.8703172